In [146]:
from torch import nn 
import pickle
import torch
gpu = torch.device("mps")

In [147]:
class TextClassificationModel(nn.Module):
    def __init__(self, x_size, hidden_layer_size, dropout, activation_fn):
        super(TextClassificationModel, self).__init__()
        self.first_layer = nn.Linear(x_size, hidden_layer_size)
        self.hidden_layer = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.output_projection_1 = nn.Linear(hidden_layer_size, 1)
        self.activation = activation_fn
        self.normalisation = nn.BatchNorm1d(hidden_layer_size)
        self.dropout = nn.Dropout1d(dropout)
        self.sigmoid = nn.Sigmoid()
        self.x_size = x_size

    def forward(self, x):
        x = self.first_layer(x)
        for i in range(14):
            x = self.hidden_layer(x)
            x = self.normalisation(x)
            x = self.activation(x)
            x = self.dropout(x)

        out = self.output_projection_1(x)
        out_distribution = self.sigmoid(out)
        return out_distribution

In [148]:
def train(
    train_dataloader, test_dataloader, loss_function, num_epochs, model, model_optimiser, batch_size
):
    # A counter for the number of gradient updates we've performed.
    num_iter = 0

    # Iterate `num_epochs` times.
    for epoch in range(num_epochs):
        print("Starting epoch {}".format(epoch + 1))
        # Iterate over the train_dataloader, unpacking the images and labels
        for data, labels in train_dataloader:
            # If we're using the GPU, move reshaped_images and labels to the GPU.
            if gpu:
                data = data.to(gpu)
                labels = labels.to(gpu)

            # Run the forward pass through the model to get predicted log distribution.
            predicted = model(data)

            # reshape
            labels = torch.unsqueeze(labels, 1)
   
            # Calculate the loss
            batch_loss = loss_function(predicted, labels)

            # Clear the gradients as we prepare to backprop.
            model_optimiser.zero_grad()

            # Backprop (backward pass), which calculates gradients.
            batch_loss.backward()

            # Take a gradient step to update parameters.
            model_optimiser.step()

            # Increment gradient update counter.
            num_iter += 1

            # Calculate test set loss and accuracy every 500 gradient updates
            # It's standard to have this as a separate evaluate function, but
            # we'll place it inline for didactic purposes.
            if num_iter % 500 == 0:
                # Set model to eval mode, which turns off dropout.
                model.eval()
                # Counters for the num of examples we get right / total num of examples.
                num_correct = 0
                total_examples = 0
                total_test_loss = 0

                with torch.no_grad():
                    # Iterate over the test dataloader
                    for test_data, test_labels in test_dataloader:

                        # If we're using the GPU, move tensors to the GPU.
                        if gpu:
                            test_data = test_data.to(gpu)
                            test_labels = test_labels.to(gpu)
                        
                        # reshape
                        test_labels = torch.unsqueeze(test_labels, 1)

                        # Run the forward pass to get predicted distribution.
                        predicted = model(test_data)
                               
                        # Calculate loss for this test batch. This is averaged, so multiply
                        # by the number of examples in batch to get a total.
                        total_test_loss += loss_function(predicted, test_labels)

                        # Get predicted labels (argmax)
                        predicted_labels = (predicted.data > 0.5).int()

                        # Count the number of examples in this batch
                        total_examples += test_labels.size(0)

                        # Count the total number of correctly predicted labels.
                        # predicted == labels generates a ByteTensor in indices where
                        # predicted and labels match, so we can sum to get the num correct.
                        num_correct += torch.sum(predicted_labels == test_labels.data)
                        
                accuracy = num_correct / total_examples
                average_test_loss = total_test_loss / total_examples
                print(
                    "Iteration {}. Test Loss {}. Test Accuracy {}. Total Examples {}".format(
                        num_iter, average_test_loss, accuracy, total_examples
                    )
                )
                # Set the model back to train mode, which activates dropout again.
                model.train()
    return model

In [149]:
# open data 
with open("./data/content_features_cv.pkl","rb") as f:
    content_features_df = pickle.load(f)
    
content_features_df.head()
print(content_features_df.shape)

(72134, 17)


In [150]:
# create dataloader 
labels = content_features_df["label"].values
features = content_features_df.drop("label",axis=1).values

features_tensor = torch.tensor(features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.float32)

dataset = torch.utils.data.TensorDataset(features_tensor,labels_tensor)
print(features_tensor.shape)
print(labels_tensor.shape)

torch.Size([72134, 16])
torch.Size([72134])


In [151]:
batch_size = 32
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset,test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [152]:
# model object
sequence_length = len(train_dataset)
print(sequence_length)
model = TextClassificationModel(
    x_size=16, hidden_layer_size=512, dropout=0.5, activation_fn=nn.ReLU()
)
loss = nn.BCELoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.005)
model.to(gpu)

57707


TextClassificationModel(
  (first_layer): Linear(in_features=16, out_features=512, bias=True)
  (hidden_layer): Linear(in_features=512, out_features=512, bias=True)
  (output_projection_1): Linear(in_features=512, out_features=1, bias=True)
  (activation): ReLU()
  (normalisation): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout1d(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)

In [153]:
num_epochs = 1
train(
    train_dataloader, 
    test_dataloader, 
    loss, 
    num_epochs, 
    model, 
    optimiser,
    batch_size
)


Starting epoch 1
Iteration 500. Test Loss 0.021658388897776604. Test Accuracy 0.5132043957710266. Total Examples 14427
Iteration 1000. Test Loss 0.02165759913623333. Test Accuracy 0.5132043957710266. Total Examples 14427
Iteration 1500. Test Loss 0.021657491102814674. Test Accuracy 0.5132043957710266. Total Examples 14427


TextClassificationModel(
  (first_layer): Linear(in_features=16, out_features=512, bias=True)
  (hidden_layer): Linear(in_features=512, out_features=512, bias=True)
  (output_projection_1): Linear(in_features=512, out_features=1, bias=True)
  (activation): ReLU()
  (normalisation): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout1d(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)